# Improving Fine-tuned Model using RAG

Code authored by: Yassine ZAIM <br>
27/09/2024

### imports

In [1]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip install docx2txt
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.3/176.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing insta

In [52]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, GPTVectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

In [4]:
from google.colab import drive
#drive.mount('/content/drive')

### Define Settings

In [65]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

Article about the selection of the embeding method for Multilingual Task: https://towardsdatascience.com/openai-vs-open-source-multilingual-embedding-models-e5ccb7c90f05

In [66]:
# articles available here: {add GitHub repo}
documents = SimpleDirectoryReader("/content/documents/").load_data()
documents

[Document(id_='1f8245c8-4aa0-40f9-ab31-33b6b1e0d7f1', embedding=None, metadata={'file_name': 'Conjoncture_T2_24_HCP.docx', 'file_path': '/content/documents/Conjoncture_T2_24_HCP.docx', 'file_type': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document', 'file_size': 365357, 'creation_date': '2024-09-27', 'last_modified_date': '2024-09-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='البحوث الفصلية حول الظرفية الاقتصادية\n\nقطاع الخدمات التجارية غير المالية\n\nقطاع تجارة الجملة  \n\n\n\nالبحوث الفصلية حول الظرفية الاقتصادية\n\nقطاع الخدمات التجارية غير المالية\n\nقطاع تجارة الجملة  \n\n\n\n\t\n\n\n\n\n\nيونيو 2024\n\n\n\n\n\nتبرز هذه المذكرة أهم ارتسامات أرباب المقاولات المستقاة من بحوث الظرفية الاقتصادية المنجزة من طرف المندوبية ا

In [67]:
for i, elm in enumerate(documents):
  print(f"The document n° {i}")
  print("===========================")
  print(elm.text)

The document n° 0
البحوث الفصلية حول الظرفية الاقتصادية

قطاع الخدمات التجارية غير المالية

قطاع تجارة الجملة  



البحوث الفصلية حول الظرفية الاقتصادية

قطاع الخدمات التجارية غير المالية

قطاع تجارة الجملة  



	





يونيو 2024





تبرز هذه المذكرة أهم ارتسامات أرباب المقاولات المستقاة من بحوث الظرفية الاقتصادية المنجزة من طرف المندوبية السامية للتخطيط برسم الفصل الثاني من سنة 2024 لدى المقاولات العاملة بقطاعي الخدمات التجارية غير المالية وتجارة الجملة. وترصد هذه الإرتسامات التطور الحاصل في إنتاج هذين القطاعين خلال الفصل الأول من سنة 2024 وكذا التوقعات الخاصة بالفصل الثاني من سنة 2024.

تبرز هذه المذكرة أهم ارتسامات أرباب المقاولات المستقاة من بحوث الظرفية الاقتصادية المنجزة من طرف المندوبية السامية للتخطيط برسم الفصل الثاني من سنة 2024 لدى المقاولات العاملة بقطاعي الخدمات التجارية غير المالية وتجارة الجملة. وترصد هذه الإرتسامات التطور الحاصل في إنتاج هذين القطاعين خلال الفصل الأول من سنة 2024 وكذا التوقعات الخاصة بالفصل الثاني من سنة 2024.











       

ارتسامات أرباب المقاول

In [68]:
# some ad hoc document refinement
print(len(documents))
for i, doc in enumerate(documents):
  if i == 20:
    print(doc.text)
    #print(doc.text[-100:])
  """    if "Member-only story" in doc.text:
          documents.remove(doc)
          continue

      if "The Data Entrepreneurs" in doc.text:
          documents.remove(doc)

      if " min read" in doc.text:
          documents.remove(doc)"""

print(len(documents))

169
RAPPORT DE SUIVI DE LA SITUATION ÉCONOMIQUE AU MAROC – LIBÉRER LE POTENTIEL DU SECTEUR PRIVÉ POUR STIMULER 
LA CROISSANCE ET LA CRÉATION D’EMPLOIS
2contraste avec des taux de croissance à deux chiffres 
de la valeur ajoutée des secteurs de l’automobile et 
de l’électronique. La contribution du secteur manufac -
turier à la croissance globale du PIB s’est nettement 
accrue au dernier trimestre de 2023 (graphique 2). La 
poursuite de la reprise du secteur de tourisme (voir 
encadré 1) a largement contribué à la bonne perfor -
mance du secteur des services, qui a progressé de 
4,4 pour cent. Du côté de la demande, la formation 
brute de capital fixe a augmenté de 1,5 pour cent, 
affichant un dynamisme croissant au cours des der -
niers mois de l’année (graphique 3). Malgré le choc inflationniste, la consommation privée a augmenté de 
3,7 pour cent en 2023.
En dépit d’un environnement 
international complexe, la 
performance à l’international 
de l’économie marocaine a été 
remarquable

In [69]:
from sentence_transformers import SentenceTransformer

# Load a multilingual model for embeddings (e.g., XLM-Roberta)
multilingual_model = SentenceTransformer('xlm-roberta-base')

# Custom embedding function for multilingual support
def embed_text(text):
    return multilingual_model.encode(text)


In [ ]:
# store docs into vector DB
#index = VectorStoreIndex.from_documents(documents)

# Alternative to use for improve retrieval for language French and Arbic
index = GPTVectorStoreIndex.from_documents(documents, embed_fn=embed_text)

### Set Up Search Function

In [ ]:
# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [ ]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [56]:
# query documents English
query = "How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure?"
response_En = query_engine.query(query)

In [59]:
# reformat response English query
context = "Context:\n"
for i in range(top_k):
    context = context + response_En.source_nodes[i].text + "\n\n"

print(context)

Context:
THE RE GIONS  37
Promoting private sector engagement and financial inclusion
The World Bank works with countries to help strengthen the private sector and 
expand access to finance. In T unisia, a $120 million project is helping small and 
medium enterprises access long-term lines of credit from the Ministry of Finance 
through participating financial institutions.
In Morocco, we are providing $450 million—the third round in a series of financ -
ing—to help the government implement reforms to improve financial inclusion,  
digital entrepreneurship, and access to digital infrastructure and services for peo-ple and businesses. The series has enabled the country to increase access to finan -
cial services to 44 percent in 2023 from 29 percent in 2017 and digital payments to 30 percent from 17 over the same period. It has also improved the infrastructure 
for digital payments, mobile payment networks, microinsurance, collateral regis -
tries, and women’s access to finance and econ

In [ ]:
# query documents French
query = "De combien a varié La valeur ajoutée totale par travailleur au Maroc?"
response_Fr = query_engine.query(query)

In [ ]:
# reformat response French query
context = "Context:\n"
for i in range(top_k):
    context = context + response_Fr.source_nodes[i].text + "\n\n"

print(context)

In [ ]:
# query documents Arabic
#query = "كيف تطور قطاع تجارة الجملة في المغرب؟"
query = " تجارة لوازم منزلية بالجملة"
response_Ar = query_engine.query(query)

In [ ]:
# reformat response Arabic query
context = "Context:\n"
for i in range(top_k):
    context = context + response_Ar.source_nodes[i].text + "\n\n"

print(context)

### Import LLM

In [64]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4713: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

ValueError: Found modules on cpu/disk. Using Exllama or Exllamav2 backend requires all the modules to be on GPU.You can deactivate exllama backend by setting `disable_exllama=True` in the quantization config object

### Use LLM

In [ ]:
# prompt (no context)
intstructions_string = f"""ZAIMYGPT, functioning as a data scientist and AI consultant. \n
He did this notebook to show its ability of using RAG solution. \n
Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

In [ ]:
comment = "How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure?"

prompt = prompt_template(comment)
print(prompt)

[INST] ZAIMYGPT, functioning as a data scientist and AI consultant. 

He did this notebook to show its ability of using RAG solution. 

Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.

Please respond to the following comment.
 
How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure? 
[/INST]


In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=500)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] ZAIMYGPT, functioning as a data scientist and AI consultant. 

He did this notebook to show its ability of using RAG solution. 

Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.

Please respond to the following comment.
 
How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure? 
[/INST] I'm glad you're interested in Morocco's role in improving financial inclusion, digital entrepreneurship, and access to digital infrastructure.

Morocco has been making significant strides in these areas. According to the World Bank, Morocco's financial inclusion rate was 63.3% in 2014, and it has been steadily increasing since then. The country's National Strategy for Digital Transition (SNTD) aims to increase financial inclusion to 75% by 2025.

In terms of digital entrepreneurship, Morocco has been investing in various initiatives to support startups and small bu

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]ZAIMYGPT, functioning as a data scientist and AI consultant.
He did this notebook to show its ability of using RAG solution.
Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.


Please respond to the following: \n
{comment} \n
Use the relevant information from the context below \n
{context}

[/INST]
"""

In [ ]:
prompt = prompt_template_w_context(context, comment)
print(prompt)

[INST]ZAIMYGPT, functioning as a data scientist and AI consultant. 
He did this notebook to show its ability of using RAG solution. 
Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.


Please respond to the following: 

How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure? 

Use the relevant information from the context below 

Context:
THE RE GIONS  37
Promoting private sector engagement and financial inclusion
The World Bank works with countries to help strengthen the private sector and 
expand access to finance. In T unisia, a $120 million project is helping small and 
medium enterprises access long-term lines of credit from the Ministry of Finance 
through participating financial institutions.
In Morocco, we are providing $450 million—the third round in a series of financ -
ing—to help the government implement reforms to improve financial inclusion,  


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=500)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST]ZAIMYGPT, functioning as a data scientist and AI consultant. 
He did this notebook to show its ability of using RAG solution. 
Its GPT improved by RAG solution will reacts to feedback aptly and ends responses with its signature '–ZAIMYGPT'.


Please respond to the following: 

How much Morocco provides to improve financial inclusion, digital entrepreneurship, and access to digital infrastructure? 

Use the relevant information from the context below 

Context:
THE RE GIONS  37
Promoting private sector engagement and financial inclusion
The World Bank works with countries to help strengthen the private sector and 
expand access to finance. In T unisia, a $120 million project is helping small and 
medium enterprises access long-term lines of credit from the Ministry of Finance 
through participating financial institutions.
In Morocco, we are providing $450 million—the third round in a series of financ -
ing—to help the government implement reforms to improve financial inclusion